In [2]:
import tensorflow as tf
from keras import regularizers
from tensorflow_model_optimization.python.core.keras.compat import keras
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_callbacks,prunable_layer,pruning_schedule,prune
from tensorflow.keras.layers import Dense,Conv2D,GlobalMaxPool2D,Dropout,Flatten
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import softmax,sigmoid,linear,relu6
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_ds = train_images / 255.0
test_ds = test_images / 255.0
batch_size = 128

In [4]:
class MyModel(Model):
    def __int__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(512, (3, 3), activation='relu6', padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01),use_bias=True,bias_initializer='he_normal',bias_regularizer=regularizers.l2(0.01))
        self.pool = GlobalMaxPool2D(strides=(2,2), padding='same')
        self.drop = Dropout(0.5)
        self.flatten = Flatten
        self.fc1 = Dense(1024, activation='relu6')
        self.fc2 = Dense(512, activation='relu6')
        self.fc3 = Dense(256, activation='relu6')
    def call(self,x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.drop(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
class SecondLayer(MyModel):
    def __init__(self):
        super(SecondLayer, self).__init__()
        self.conv2 = Conv2D(512, (3, 3), activation='relu6', padding='same',kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(0.01),use_bias=True,bias_initializer='he_normal',bias_regularizer=regularizers.l2(0.01))
        self.pool2 = GlobalMaxPool2D(strides=(2,2), padding='same')
        self.drop2 = Dropout(0.5)
        self.flatten2 = Flatten
        self.fc4 = Dense(1024, activation='relu6')
        self.fc6 = Dense(512, activation='relu6')
        self.fc7 = Dense(256, activation='relu6')

    def call(self,x):
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.drop2(x)
        x = self.flatten2(x)
        x = self.fc4(x)
        x = self.fc6(x)
        x = self.fc7(x)
        return x
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='sum_over_batch_size',axis=-1)
optimizer = tf.keras.optimizers.Adam(learning_rate=10e-5)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [6]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training = True)
        loss = loss_object(labels, predictions)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss.update_state(loss)
        train_accuracy.update_state(labels, predictions)

@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss.update_state(t_loss)
  test_accuracy.update_state(labels, predictions)

EPOCHS = 10

for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images,labels)

    for images, labels in test_ds:
        test_step(images,labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result():0.2f}, '
        f'Accuracy: {train_accuracy.result() * 100:0.2f}, '
        f'Test Loss: {test_loss.result():0.2f}, '
        f'Test Accuracy: {test_accuracy.result() * 100:0.2f}'
    )

ValueError: too many values to unpack (expected 2)